<p style='direction:rtl'>
در این فایل با استفاده از کتابخانه های beautifulsoup و requests دو تابع get_urls و get_page_data نوشته شده است.
تابع get_urls برای گرفتن لینک صفحه هایی است که اطلاعات آن جمع‌آوری شده است.
تابع get_pages_data برای استخراج اطلاعات لینک هایی که به صورت یک لیست به آن داده می شود، استفاده می شود.
</p>

In [ ]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import pandas as pd

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
def get_urls(input_urls: list)-> list:
  output_urls = []

  for url in input_urls:
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'lxml')
    urls_section_div = soup.find_all('div', class_='mw-category mw-category-columns')
    a_tags = urls_section_div[1].find_all('a')

    for a in a_tags:
      output_urls.append('https://fa.wikipedia.org' + a.get('href'))


  return output_urls

In [ ]:
def get_pages_data(urls: list)-> list:
  pages_data = []

  for url in tqdm(urls):
      html_text = requests.get(url).text
      soup = BeautifulSoup(html_text, 'lxml')
      div_content = soup.find('div', class_='mw-parser-output')
      p_tags = div_content.find_all('p')

      for paragraph in p_tags:
        text = paragraph.get_text(separator=' ', strip = True)
        if len(text) > 0:
          pages_data.append(text)

  return pages_data

In [ ]:
source_urls = ['https://fa.wikipedia.org/wiki/%D8%B1%D8%AF%D9%87:%D8%A8%DB%8C%D9%88%D8%A7%D9%86%D9%81%D9%88%D8%B1%D9%85%D8%A7%D8%AA%DB%8C%DA%A9',
               'https://fa.wikipedia.org/wiki/%D8%B1%D8%AF%D9%87:%DA%98%D9%86%D8%AA%DB%8C%DA%A9',
               'https://fa.wikipedia.org/w/index.php?title=%D8%B1%D8%AF%D9%87:%DA%98%D9%86%D8%AA%DB%8C%DA%A9&pagefrom=%DA%AF%D8%B1%D8%A7%DB%8C%D8%B4+%D8%AC%D9%86%D8%B3%DB%8C+%D9%88+%D8%B2%DB%8C%D8%B3%D8%AA%E2%80%8C%D8%B4%D9%86%D8%A7%D8%B3%DB%8C#mw-pages',
               'https://fa.wikipedia.org/wiki/%D8%B1%D8%AF%D9%87:%D8%B2%DB%8C%D8%B3%D8%AA%E2%80%8C%D8%B4%D9%86%D8%A7%D8%B3%DB%8C_%D9%85%D9%88%D9%84%DA%A9%D9%88%D9%84%DB%8C',
               'https://fa.wikipedia.org/w/index.php?title=%D8%B1%D8%AF%D9%87:%D8%B2%DB%8C%D8%B3%D8%AA%E2%80%8C%D8%B4%D9%86%D8%A7%D8%B3%DB%8C_%D9%85%D9%88%D9%84%DA%A9%D9%88%D9%84%DB%8C&pagefrom=%D9%86%D9%82%D8%B4%D9%87+%DA%98%D9%86#mw-pages'
               ]

urls = get_urls(source_urls)

In [ ]:
len(urls)

609

In [ ]:
df_urls = pd.DataFrame(urls)

In [ ]:
df_urls.to_csv('urls.csv')

In [ ]:
data = get_pages_data(urls)

100%|██████████| 609/609 [03:16<00:00,  3.11it/s]


In [ ]:
df = pd.DataFrame(data=data,columns=['text'])

In [ ]:
df.to_csv('wiki_bio_unclean.csv')